In [12]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 8
batch_size = 4
max_iters = 2000
learning_rate = 3e-4
eval_iters = 250
dropout = 0.2

cuda


In [4]:
with open ('wizard_of_oz.txt','r',encoding = 'utf-8') as f:
    text = f.read()
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)
print(vocab_size)


['\n', ' ', '!', '"', '#', '&', "'", '(', ')', '*', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']
83


In [5]:
string_to_int = {ch:i for i,ch in enumerate(chars)}
int_to_string = {i:ch for i,ch in enumerate(chars)}
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text),dtype = torch.long)
print(data[:100])

tensor([82, 46, 63, 60,  1, 42, 73, 70, 65, 60, 58, 75,  1, 33, 76, 75, 60, 69,
        57, 60, 73, 62,  1, 60, 28, 70, 70, 66,  1, 70, 61,  1, 30, 70, 73, 70,
        75, 63, 80,  1, 56, 69, 59,  1, 75, 63, 60,  1, 49, 64, 81, 56, 73, 59,
         1, 64, 69,  1, 41, 81,  0,  1,  1,  1,  1,  0, 46, 63, 64, 74,  1, 60,
        57, 70, 70, 66,  1, 64, 74,  1, 61, 70, 73,  1, 75, 63, 60,  1, 76, 74,
        60,  1, 70, 61,  1, 56, 69, 80, 70, 69])


In [6]:
n = int(0.8*len(data))
train_data = data[:n]
val_data =  data[n:]

print(len(train_data))
print(len(val_data))

def get_batch(split):
    data = train_data if split == 'train' else val_data
    #print(len(data)-block_size)
    ix = torch.randint(len(data)-block_size,(batch_size,))
    #print(ix)
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y


x, y = get_batch('train')
print('inputs:')
print(x)
print('targets:')
print(y)

186744
46686
inputs:
tensor([[64, 75, 60,  1, 56,  1, 78, 63],
        [ 1, 70, 61,  1, 74, 75, 56, 75],
        [ 3,  1, 56, 74, 66, 60, 59,  1],
        [73, 70, 58, 70, 59, 64, 67, 60]], device='cuda:0')
targets:
tensor([[75, 60,  1, 56,  1, 78, 63, 64],
        [70, 61,  1, 74, 75, 56, 75, 60],
        [ 1, 56, 74, 66, 60, 59,  1, 75],
        [70, 58, 70, 59, 64, 67, 60, 74]], device='cuda:0')


In [7]:

x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print("when input is",context,"target is",target)

when input is tensor([82]) target is tensor(46)
when input is tensor([82, 46]) target is tensor(63)
when input is tensor([82, 46, 63]) target is tensor(60)
when input is tensor([82, 46, 63, 60]) target is tensor(1)
when input is tensor([82, 46, 63, 60,  1]) target is tensor(42)
when input is tensor([82, 46, 63, 60,  1, 42]) target is tensor(73)
when input is tensor([82, 46, 63, 60,  1, 42, 73]) target is tensor(70)
when input is tensor([82, 46, 63, 60,  1, 42, 73, 70]) target is tensor(65)


In [8]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train','val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X,Y = get_batch(split)
            logits,loss = model (X,Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [9]:
class BigramLanguageModel(nn.Module):
    def __init__(self,vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size,vocab_size)

    def forward(self,index,targets = None):
        logits = self.token_embedding_table(index)
        
       

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T,C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits,targets)
            
            

        return logits, loss 

    def generate(self,index,max_new_tokens):
        for i in range(max_new_tokens):
            logits , loss = self.forward(index)

            logits = logits[:,-1,:]

            probs = F.softmax(logits,dim=-1)

            index_next = torch.multinomial(probs,num_samples = 1)

            index = torch.cat((index,index_next),dim=1)
        return index


model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1) , dtype= torch.long,device = device)
generated_chars = decode(m.generate(context,max_new_tokens=500)[0].tolist())
print(generated_chars)



7]q_V'ORgw:S?sVD*LRZQUMkMhuJ6u;]#RW4'(zIDI.#lAxW4F;)﻿eZsdF'OGfn5J')k("
]&BxZ
qs3GRj:PJPDg1.F4olURrpzfQj3;:P﻿06EK**
(?Bdfi2-rp_G3'7a﻿7t
G4lMx](oN?T1jkr/"j9E[NGh(Rmk93gqWcXE"4ax'8M&YVfkn#.6LM1*3zvpO2tM:5aq!/CCebeF;JXSJ9/8WM8A8P5t*4KPO;H! *"/s
Kd3PR_mR FboSAI hKeWq(8fO:4DoVTw/,Hn[E,u?]FgSH_
_mmRdYo9,KH*dL/"r8GH(tCZN5cqH72.AeeSjO:"& j?0)f]OXb5Wqx'mI)jwbVE""
EKBdi2MvGO'p;3f&G_TmaHkIMx!CdQUuo?.3:bKb]dnT)
R'OXfP,/;);:y﻿Nh3Y4K-pBQ)!EKJk"mNw
_OuMh3kSy(riG_Adctro8fa.s'aZrMcax﻿ew!hkE5ORVJi.Fs-cXO(Ah.5QbY((


In [13]:
optimizer = torch.optim.AdamW(model.parameters(),lr = learning_rate)

for iter in range(max_iters):
    
   
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train_loss: {losses['train']:.3f} , val_loss: {losses['val']:.3f}")

    xb,yb = get_batch('train')

    logits,loss = model.forward(xb,yb)
    optimizer.zero_grad(set_to_none = True)
    loss.backward()
    optimizer.step()

    



print(loss.item())


step: 0, train_loss: 4.120 , val_loss: 4.138
step: 250, train_loss: 4.067 , val_loss: 4.086
step: 500, train_loss: 4.028 , val_loss: 4.025
step: 750, train_loss: 3.975 , val_loss: 3.967
step: 1000, train_loss: 3.954 , val_loss: 3.951
step: 1250, train_loss: 3.869 , val_loss: 3.903
step: 1500, train_loss: 3.834 , val_loss: 3.862
step: 1750, train_loss: 3.778 , val_loss: 3.822
3.3479244709014893


In [14]:
context = torch.zeros((1,1) , dtype= torch.long,device = device)
generated_chars = decode(m.generate(context,max_new_tokens=500)[0].tolist())
print(generated_chars)


i2q8cIfwC(emp/Ji.DK)]G"3vyOwT3T/nj9ceeFYk!go&H8G:"drzAhHCh(riwevq1Maqm?v'tacqt FvpwRz( fAdQBR4#akXZx8P,
qakz(uwGDQ[7 in.knAPF;26,ck?!LZ)ji1,q!"zT9t!MouYo*'EtfDEKJ'cO(un-kFNh3"sun20D9_[D2o0PgiGJkSB?Mx'wrod﻿2173/;n;(6ixZW!nTWkn mvedt *;A& a h(geTK;jn[_RKun.*btbOk(67E5R:X#xZhJoba d
uMO#T3:Z,2#_R904FsI]:,n!
wI)Kf. hcZV'Ezvg a eOl.R4t,/KXVzJ).NGyTR4GxZO.IjCSYHS#(R?g R

M"rZe nTN(I9K_Rro*KG'H:KFov6ik_F;
B8Pj*L_aoljLZ9ly_:otvVa.#7fC
nfiu.]U:kJpgm"r*i
beejnaQ!xI.PTBy 57f3uM/rZrvW#9bh(lS&vTcpZ-Qfad Yo:.U
